In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from scipy import ndimage
from keras.optimizers import Adadelta,Adam
from keras.callbacks import TensorBoard, CSVLogger, ModelCheckpoint
# from lipnet.lipreading.generators import BasicGenerator
# from lipnet.lipreading.callbacks import Statistics, Visualize
# from lipnet.lipreading.curriculums import Curriculum
# from lipnet.core.decoders import Decoder
# from lipnet.lipreading.helpers import labels_to_text
# from lipnet.utils.spell import Spell
from lipnet.model2 import LipNet
import numpy as np
import datetime
import cv2
import skvideo.io
import dlib
from scipy.misc import imresize
# from lipnet.lipreading.aligns import Align
import imageio
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
import random
import tensorflow as tf
import keras.backend as K
tf_config = tf.ConfigProto(allow_soft_placement=False)
tf_config.gpu_options.allow_growth = True
s = tf.Session(config=tf_config)
K.set_session(s)

FACE_PREDICTOR_PATH  = os.path.join('common','predictors','shape_predictor_68_face_landmarks.dat')
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(FACE_PREDICTOR_PATH)

frames_count = 30
words_folder = 'training/unseen_speakers/TARGET/datasets/lipread_mp4'
words = os.listdir(words_folder)
# words = ['EVERYONE',
#  'EVENTS',
#  'EVERYBODY',
#  'CLOUD',
#  'RIGHTS',
#  'AUTHORITIES',
#  'WITHIN',
#  'SEVERAL',
#  'ABSOLUTELY',
#  'WINDS',
#  'AFTERNOON',
#  'BETWEEN',
#  'TALKING',
#  'AGREEMENT',
#  'PRICES'] #words[:2]
words = [s.upper() for s in words]
max_len = 12 #np.max([len(word) for word in words])
used_words = {''}
print(words)
def datagen(data_partition = 'train', batch_size = 16):
    while True:
        counter = 0
        
        X_data = []#np.zeros((batch_size,))
        Y_data = []
        input_length = []
        label_length = []
        source_str = []
        while counter<batch_size:
#             print(counter)
            word_idx = np.random.randint(len(words))
            used_words.add(words[word_idx])
            train_folder = 'training/unseen_speakers/TARGET/datasets/lipread_mp4/{0}/{1}/'.format(words[word_idx],data_partition)
        #         print(train_folder)
            files = os.listdir(train_folder)
        #         print(files)
            paths = list(set([x.split('.')[0] for x in files]))
        
            idx = np.random.randint(len(paths))
            path = os.path.join(train_folder,paths[idx])
            frames = []
            for img_name in sorted(os.listdir(path)):
                img = cv2.imread(os.path.join(path,img_name))
                frames.append(img)
            frames = np.array(frames)
            if len(frames.shape)==4:
                frames = np.rollaxis(frames,2, start=1)
    #             print(path)
    #             print(frames.shape)
    #             print('---------')
                if 'numpy.ndarray' in str(type(frames)) \
                        and len(np.shape(frames))==4 \
                        and np.shape(frames)[1] == 100 \
                        and np.shape(frames)[2] == 50:

                    X_data.append(frames)


                    ####
                    Y_data.append([ord(c) - 65 for c in words[word_idx]]+[-1]*(max_len-len(words[word_idx])))
                    input_length.append(frames_count)
                    label_length.append(len(words[word_idx]))
                    source_str.append(words[word_idx])
                    counter+=1
                else:
                    continue
            else:
                print('error:', frames.shape, path)
                continue

        X_data = np.array(X_data).astype(np.float32) / 255 
        Y_data = np.array(Y_data)
        input_length = np.array(input_length)
        label_length = np.array(label_length)
        source_str = np.array(source_str)
        
        inputs = {'the_input': X_data,
                  'the_labels': Y_data,
                  'input_length': input_length,
                  'label_length': label_length,
                  'source_str': source_str  # used for visualization only
                  }
        outputs = {'ctc': np.zeros([batch_size])}
        
#         print('the_input:', inputs['the_input'].shape)
#         print('the_labels:', inputs['the_labels'].shape)
#         print('input_length:', inputs['input_length'].shape)
#         print('label_length:', inputs['label_length'].shape)
#         print('source_str:', inputs['source_str'].shape)
#         print('batch_frames_shape',X_data.shape, data_partition)
        yield (inputs, outputs)

['EVERYONE', 'EVENTS', 'EVERYBODY', 'CLOUD', 'RIGHTS', 'AUTHORITIES', 'WITHIN', 'SEVERAL', 'ABSOLUTELY', 'WINDS', 'AFTERNOON', 'BETWEEN', 'TALKING', 'AGREEMENT', 'PRICES', 'MEETING', 'AFFAIRS', 'SERIOUS', 'SUNSHINE', 'OPPOSITION']


In [3]:
# global lip_gen

run_name = datetime.datetime.now().strftime('%Y:%m:%d_%H:%M:%S')
run_name, start_epoch, stop_epoch, img_c, img_w, img_h, frames_n, absolute_max_string_len, minibatch_size = \
    run_name, 0, 1000, 3, 100, 50, 30, 12, 2



CURRENT_PATH = os.getcwd()+'/training/unseen_speakers'
DATASET_DIR  = os.path.join(CURRENT_PATH, 'datasets')
OUTPUT_DIR   = os.path.join(CURRENT_PATH, 'results')
LOG_DIR      = os.path.join(CURRENT_PATH, 'logs')
# if not os.path.exists('training/unseen_speakers/results/'+run_name):
#     os.makedirs('training/unseen_speakers/results/'+run_name)
if not os.path.exists('training/unseen_speakers/logs/'+run_name):
    os.makedirs('training/unseen_speakers/logs/'+run_name)

PREDICT_GREEDY      = False
PREDICT_BEAM_WIDTH  = 200
PREDICT_DICTIONARY  = os.path.join(CURRENT_PATH,'..','..','common','dictionaries','lfw.txt')
FACE_PREDICTOR_PATH  = os.path.join(CURRENT_PATH,'..','..','common','predictors','shape_predictor_68_face_landmarks.dat')

# def curriculum_rules(epoch):
#     return { 'sentence_length': -1, 'flip_probability': 0.5, 'jitter_probability': 0.05 }


# # def train(run_name, start_epoch, stop_epoch, img_c, img_w, img_h, frames_n, absolute_max_string_len, minibatch_size):
# curriculum = Curriculum(curriculum_rules)
# lip_gen = BasicGenerator(dataset_path=DATASET_DIR, #vtype = "face", face_predictor_path = FACE_PREDICTOR_PATH,
#                             minibatch_size=minibatch_size,
#                             img_c=img_c, img_w=img_w, img_h=img_h, frames_n=frames_n,
#                             absolute_max_string_len=absolute_max_string_len,
#                             curriculum=curriculum, start_epoch=start_epoch).build()
# lip_gen_train = datagen(data_partition='train', batch_size=5)
# lip_gen_val = datagen(data_partition='val', batch_size=5)

lipnet = LipNet(img_c=img_c, img_w=img_w, img_h=img_h, frames_n=frames_n,
                        absolute_max_string_len=absolute_max_string_len, output_size=27)
# lipnet.summary()
# adadelta = Adadelta(lr=0.0001, decay=1e-08, epsilon=1e-08)

adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=1e-08)

# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
lipnet.model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam, metrics=['accuracy'])


# spell = Spell(path=PREDICT_DICTIONARY)
# decoder = Decoder(greedy=PREDICT_GREEDY, beam_width=PREDICT_BEAM_WIDTH,
#                   postprocessors=[labels_to_text, spell.sentence])

# define callbacks
# statistics  = Statistics(lipnet, lip_gen.next_val(), decoder, 256, output_dir=os.path.join(OUTPUT_DIR, run_name))
# visualize   = Visualize(os.path.join(OUTPUT_DIR, run_name), lipnet, lip_gen.next_val(), decoder, num_display_sentences=minibatch_size)
tensorboard = TensorBoard(log_dir=os.path.join(LOG_DIR, 'crop-'+run_name))
# csv_logger  = CSVLogger(os.path.join(LOG_DIR, "{}-{}.csv".format('training',run_name)), separator=',', append=True)
checkpoint  = ModelCheckpoint(save_best_only=True,
                              filepath='training/unseen_speakers/results/weights_lfw-crop-v2.hdf5'
                              , monitor='val_loss'
                              , verbose=1
                              , mode='min'
                              , period=1)
    
    # x = next(lip_gen.next_val())
    # for i in x:
    #     print('$$$$$----------', type(i))





y_pred (?, 30, 27)


In [4]:
# run_name = datetime.datetime.now().strftime('%Y:%m:%d:%H:%M:%S')
# train(run_name, 0, 1000, 3, 100, 50, 75, 32, 2)

In [5]:
# os.getcwd()+'training/unseen_speakers'

In [6]:
# g=lip_gen.next_train()
# inputs, outputs = next(g)

In [7]:
# inputs

In [8]:
# len('place green at t six now')

In [9]:
# len([ 15.,  11.,   0.,   2.,   4.,  26.,   6.,  17.,   4.,   4.,  13.,
#           26.,   0.,  19.,  26.,  19.,  26.,  18.,   8.,  23.,  26.,  13.,
#           14.,  22.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.])

In [10]:
# words

In [21]:
# load weight if necessary

start_epoch = 100000
if start_epoch > 0:
    weight_file = os.path.join(OUTPUT_DIR, 'weights_lfw-crop-v2.hdf5')
    lipnet.model.load_weights(weight_file)

lipnet.model.fit_generator(generator=datagen(data_partition='train', batch_size=32),
                    steps_per_epoch=1
                    , epochs=100100,
                    #steps_per_epoch=lip_gen.default_training_steps, epochs=stop_epoch,
                    validation_data=datagen(data_partition='val', batch_size=32)
                    , validation_steps=1,#validation_steps=lip_gen.default_validation_steps,
                    callbacks=[checkpoint,tensorboard],                         
                    initial_epoch=start_epoch, 
                    verbose=0)
                    #max_q_size=5,
                    #workers=2,
                    #pickle_safe=True)


Epoch 100001: val_loss did not improve from 0.00208

Epoch 100002: val_loss did not improve from 0.00208

Epoch 100003: val_loss did not improve from 0.00208

Epoch 100004: val_loss did not improve from 0.00208

Epoch 100005: val_loss did not improve from 0.00208

Epoch 100006: val_loss did not improve from 0.00208

Epoch 100007: val_loss did not improve from 0.00208

Epoch 100008: val_loss did not improve from 0.00208

Epoch 100009: val_loss did not improve from 0.00208

Epoch 100010: val_loss did not improve from 0.00208

Epoch 100011: val_loss did not improve from 0.00208

Epoch 100012: val_loss did not improve from 0.00208

Epoch 100013: val_loss did not improve from 0.00208

Epoch 100014: val_loss did not improve from 0.00208

Epoch 100015: val_loss did not improve from 0.00208

Epoch 100016: val_loss did not improve from 0.00208

Epoch 100017: val_loss did not improve from 0.00208

Epoch 100018: val_loss did not improve from 0.00208

Epoch 100019: val_loss did not improve from 0

In [12]:
# g=datagen(data_partition='test', batch_size=1)
# x,y = next(g)

In [13]:
# used_words

In [14]:
# x['the_input'][0].shape, x['source_str'], x['the_labels']


In [15]:
# # im = batch_frames[1][25]
# import matplotlib.pyplot as plt
# %matplotlib inline
# # generator = datagen(min_row_number=0,max_row_number=200000, batch_size= 6)
# # X,Y = next(generator)
# X = x['the_input'][0]
# # print(X.shape, Y.shape)
# def plot_data(x,ax):
# #     x = x.reshape((size,size))
#     ax.imshow(x, cmap='gray')
# #     if y is not None:
# #         ax.scatter(y[0::2] , y[1::2] , marker='x', s=10)
# fig = plt.figure(figsize=(30,30))
# fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
# for i in range(30):
#     ax = fig.add_subplot(1, 30, i + 1, xticks=[], yticks=[])
#     plot_data(X[i], ax)

In [16]:
# # import io
# # import base64
# # from IPython.display import HTML

# # video = io.open('evaluation/samples/test.mp4', 'r+b').read()
# # encoded = base64.b64encode(video)
# # HTML(data='''<video alt="test" controls>
# #                 <source src="data:video/mp4;base64,{0}" type="video/mp4" />
# #              </video>'''.format(encoded.decode('ascii')))
# im = cv2.imread('training/unseen_speakers/test_pic.jpg')
# ims = get_frames_mouth(detector, predictor, [im])
# plt.imshow(ims[0][...,::-1])

In [17]:
# # np.max([len(word) for word in words])
# type(np.array([1,2]))

In [18]:
# lipnet.predict(np.array([X]))

In [19]:
# m = []
# m.append([[1,1],[1,1],[1,1]])
# m.append([[1,1],[1,1],[1,1]])
# m.append([[1,1],[1,1],[1,1]])
# m.append([[1,1],[1,1],[1,1]])
# m.append([[1,1],[1,1],[1,1]])
# m.append(1)
# # m.append([1,1,1])
# # m.append([1,1,1])
# # m.append([1,1,1])
# # m.append([1,1,1])
# # m.append([[1],[1],[1]])
# np.array(m).shape
t = {1,2,3,4,5}
t.add(5)
t.add(6)

In [20]:
t

{1, 2, 3, 4, 5, 6}